### Mean Reversion Strategy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from backtesting import backtest_strategy
from backtesting import generate_atr

ImportError: cannot import name 'generate_atr' from 'backtesting' (c:\Users\Matthew S\Documents\GitHub\mean-reversion-trading-bot\backtesting.py)

In [7]:
def fetch_stock_data(ticker, startdate, enddate):
    data = yf.download(ticker, start = startdate, end = enddate)
    data = data.reset_index()
    #rename columns
    data = data.rename(columns= {'Date': 'date', "Open": "open", "Close": "close", "High": "high", "Low": "low", "Volume": "volume"})
    return data

In [17]:
def generate_signals(data):
    bollinger_period = 20
    data["sma_20"] = data["close"].rolling(bollinger_period).mean()
    data["stdev_20"] = data["close"].rolling(bollinger_period).std()

    #Bollinger bands
    data["upper_bollinger"] = data["sma_20"] + 2 * data["stdev_20"]
    data["lower_bollinger"] = data["sma_20"] - 2 * data["stdev_20"]

    #RSI
    rsi_period = 14
    data["delta"] = data["close"].diff()
    data["gain"] = data["delta"].clip(lower = 0)
    data["loss"] = data["delta"].clip(upper = 0) * -1
    data["avg_gain"] = data["gain"].ewm(span = rsi_period).mean()
    data["avg_loss"] = data["loss"].ewm(span = rsi_period).mean()
    data["rsi_index"] = 100 - (100 / (1 + data["avg_gain"] / data["avg_loss"]))

    #sanity check
    print(data.head())
    print(data.tail())
    
    #Signals
    data["signal"] = 0

    long_condition = (data["rsi_index"] < 30) & (data["close"] < data["lower_bollinger"])
    short_condition = (data["rsi_index"] > 70) & (data["close"] > data["upper_bollinger"])
 
    data.loc[long_condition, "signal"] = 1
    data.loc[short_condition, "signal"] = -1

    data.loc[data["sma_20"].isna(), "signal"] = 0 
    data.loc[data["atr"].isna(), "signal"] = 0
    data.loc[data["rsi_index"].isna(), "signal"] = 0

    return data

In [16]:
tests = [
    {"ticker": "UNH", "start": "2019-01-01", "end": "2021-12-31"},
    {"ticker": "TSLA", "start": "2020-01-01", "end": "2022-12-31"},
    {"ticker": "SPY", "start": "2022-01-01", "end": "2023-12-31"},
]

for test in tests:
    ticker, start, end = test["ticker"], test["start"], test["end"]
    data = fetch_stock_data(ticker, start, end)
    data = generate_signals(data)

    #sanity check plot
    plt.figure(figsize=(12,5))
    plt.xticks(rotation=45)

    plt.plot(data['date'], data['close'], label = 'Close')
    plt.plot(data['date'], data['sma_20'], label = '20 Ma')
    plt.fill_between(data['date'], data['upper_bollinger'], data['lower_bollinger'], label = 'Bollinger Bands', color='lightgrey')
    plt.title(ticker, fontweight = 'bold')

    plt.legend()        
    plt.show()

C:\Users\Matthew S\AppData\Local\Temp\ipykernel_28772\3810399447.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start = startdate, end = enddate)
[*********************100%***********************]  1 of 1 completed


Price        date       close        high         low        open   volume  \
Ticker                    UNH         UNH         UNH         UNH      UNH   
0      2019-01-02  219.967056  222.550760  218.142199  221.331174  4063600   
1      2019-01-03  213.968506  219.957988  213.318062  219.957988  4623200   
2      2019-01-04  216.470917  219.939955  215.730142  216.633535  5367600   
3      2019-01-07  216.886475  218.602915  215.215193  217.175553  4133000   
4      2019-01-08  219.786423  221.141512  216.552289  218.919173  3618600   

Price  sma_20 stdev_20 upper_bollinger lower_bollinger     delta      gain  \
Ticker                                                                       
0         NaN      NaN             NaN             NaN       NaN       NaN   
1         NaN      NaN             NaN             NaN -5.998550  0.000000   
2         NaN      NaN             NaN             NaN  2.502411  2.502411   
3         NaN      NaN             NaN             NaN  0.41555

ValueError: Data must be 1-dimensional, got ndarray of shape (756, 756) instead